# PROBLEM STATEMENT#

Given the attributes of various customers of a bank, we need to build a model that can help predict if a customer's average balance will fall below minimum balance in the upcomimg quarter, based on current activity;

**FEATURES IN DATASET**

1. **Customer_ID** - ID of customer          
2. **Vintage** -  vintage of the customer with the bank, in number of days                            
3. **Age**  -  age of customer                            
4. **Gender** - gender of customer                            
5. **Dependents** - number of dependents
6. **Occupation** - occupation of customer
7. **City** - city of customer (integer coded) 
8. **customer_nw_category** -  net worth of customer (integer coded)
9. **branch code** - branch code for customer account
10. **days_since_last_transaction** - no of days since last credit in last 1 year
11. **current_balance** - balance as of today
12. **previous_month_end_balance** - end of balance previous month
13. **average_month_end_balance_prevQ** - average monthly balance in previous quarter
14. **average_month_end_balance_prevQ2** - average monthly balance in previous to previous quarter
15. **current_month_credit** - total credit amount current month
16. **previous_month_credit** - total credit amount previous month
17. **current_month_debit** - total debit amount current month
18. **previous_month_debit** - total debut amount previous month
19. **current_month_balance** - average balance of current month
20. **previous_month_balance** - average balance of previous month
21. **churn** - average balance of customer falls below minimum balance in next quarter (1 : yes, 0 : no)


In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from IPython.display import display_html

from scipy import stats

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler


from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score


from sklearn import metrics

from xgboost import XGBRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier


import warnings
warnings.filterwarnings('always')
warnings.filterwarnings('ignore')

In [2]:
customer_data = pd.read_csv(r"C:\Users\Sasidharan\Desktop\Data Science Projects\Customer Churn Prediction\churn_prediction.csv")

**Exploratory Data Analysis (EDA) **

In [3]:
customer_data.shape

(28382, 21)

In [4]:
customer_data.head()

,customer_id,vintage,age,gender,dependents,occupation,city,customer_nw_category,branch_code,days_since_last_transaction,...,previous_month_end_balance,average_monthly_balance_prevQ,average_monthly_balance_prevQ2,current_month_credit,previous_month_credit,current_month_debit,previous_month_debit,current_month_balance,previous_month_balance,churn
0,1,3135,66,Male,0.0,self_employed,187.0,2,755,224.0,...,1458.71,1458.71,1449.07,0.20,0.20,0.20,0.20,1458.71,1458.71,0
1,2,310,35,Male,0.0,self_employed,NaN,2,3214,60.0,...,8704.66,7799.26,12419.41,0.56,0.56,5486.27,100.56,6496.78,8787.61,0
2,4,2356,31,Male,0.0,salaried,146.0,2,41,NaN,...,5815.29,4910.17,2815.94,0.61,0.61,6046.73,259.23,5006.28,5070.14,0
3,5,478,90,NaN,NaN,self_employed,1020.0,2,582,147.0,...,2291.91,2084.54,1006.54,0.47,0.47,0.47,2143.33,2291.91,1669.79,1
4,6,2531,42,Male,2.0,self_employed,1494.0,3,388,58.0,...,1401.72,1643.31,1871.12,0.33,714.61,588.62,1538.06,1157.15,1677.16,1


In [5]:
customer_data.dtypes

customer_id                         int64
vintage                             int64
age                                 int64
gender                             object
dependents                        float64
occupation                         object
city                              float64
customer_nw_category                int64
branch_code                         int64
days_since_last_transaction       float64
current_balance                   float64
previous_month_end_balance        float64
average_monthly_balance_prevQ     float64
average_monthly_balance_prevQ2    float64
current_month_credit              float64
previous_month_credit             float64
current_month_debit               float64
previous_month_debit              float64
current_month_balance             float64
previous_month_balance            float64
churn                               int64
dtype: object

In [6]:
customer_data.describe()

,customer_id,vintage,age,dependents,city,customer_nw_category,branch_code,days_since_last_transaction,current_balance,previous_month_end_balance,average_monthly_balance_prevQ,average_monthly_balance_prevQ2,current_month_credit,previous_month_credit,current_month_debit,previous_month_debit,current_month_balance,previous_month_balance,churn
count,28382.000000,28382.000000,28382.000000,25919.000000,27579.000000,28382.000000,28382.000000,25159.000000,2.838200e+04,2.838200e+04,2.838200e+04,2.838200e+04,2.838200e+04,2.838200e+04,2.838200e+04,2.838200e+04,2.838200e+04,2.838200e+04,28382.000000
mean,15143.508667,2364.336446,48.208336,0.347236,796.109576,2.225530,925.975019,69.997814,7.380552e+03,7.495771e+03,7.496780e+03,7.124209e+03,3.433252e+03,3.261694e+03,3.658745e+03,3.339761e+03,7.451133e+03,7.495177e+03,0.185329
std,8746.454456,1610.124506,17.807163,0.997661,432.872102,0.660443,937.799129,86.341098,4.259871e+04,4.252935e+04,4.172622e+04,4.457581e+04,7.707145e+04,2.968889e+04,5.198542e+04,2.430111e+04,4.203394e+04,4.243198e+04,0.388571
min,1.000000,180.000000,1.000000,0.000000,0.000000,1.000000,1.000000,0.000000,-5.503960e+03,-3.149570e+03,1.428690e+03,-1.650610e+04,1.000000e-02,1.000000e-02,1.000000e-02,1.000000e-02,-3.374180e+03,-5.171920e+03,0.000000
25%,7557.250000,1121.000000,36.000000,0.000000,409.000000,2.000000,176.000000,11.000000,1.784470e+03,1.906000e+03,2.180945e+03,1.832507e+03,3.100000e-01,3.300000e-01,4.100000e-01,4.100000e-01,1.996765e+03,2.074408e+03,0.000000
50%,15150.500000,2018.000000,46.000000,0.000000,834.000000,2.000000,572.000000,30.000000,3.281255e+03,3.379915e+03,3.542865e+03,3.359600e+03,6.100000e-01,6.300000e-01,9.193000e+01,1.099600e+02,3.447995e+03,3.465235e+03,0.000000
75%,22706.750000,3176.000000,60.000000,0.000000,1096.000000,3.000000,1440.000000,95.000000,6.635820e+03,6.656535e+03,6.666887e+03,6.517960e+03,7.072725e+02,7.492350e+02,1.360435e+03,1.357553e+03,6.667958e+03,6.654693e+03,0.000000
max,30301.000000,12899.000000,90.000000,52.000000,1649.000000,3.000000,4782.000000,365.000000,5.905904e+06,5.740439e+06,5.700290e+06,5.010170e+06,1.226985e+07,2.361808e+06,7.637857e+06,1.414168e+06,5.778185e+06,5.720144e+06,1.000000


In [7]:
customer_data.isnull().sum()

customer_id                          0
vintage                              0
age                                  0
gender                             525
dependents                        2463
occupation                          80
city                               803
customer_nw_category                 0
branch_code                          0
days_since_last_transaction       3223
current_balance                      0
previous_month_end_balance           0
average_monthly_balance_prevQ        0
average_monthly_balance_prevQ2       0
current_month_credit                 0
previous_month_credit                0
current_month_debit                  0
previous_month_debit                 0
current_month_balance                0
previous_month_balance               0
churn                                0
dtype: int64

**Gender**

In [8]:
customer_data["gender"].value_counts()

Male      16548
Female    11309
Name: gender, dtype: int64

In [9]:
# Creating a third category;
customer_data["gender"].fillna("other",inplace=True)

**Dependents,Occupation,City**

In [10]:
# All these are categorical variables, we can use mode imputation;
customer_data["dependents"].value_counts()

0.0     21435
2.0      2150
1.0      1395
3.0       701
4.0       179
5.0        41
6.0         8
7.0         3
36.0        1
52.0        1
25.0        1
9.0         1
50.0        1
32.0        1
8.0         1
Name: dependents, dtype: int64

In [11]:
customer_data["city"].value_counts()

1020.0    3479
1096.0    2016
409.0     1334
146.0     1291
834.0     1138
          ... 
629.0        1
527.0        1
1212.0       1
530.0        1
70.0         1
Name: city, Length: 1604, dtype: int64

In [12]:
customer_data["occupation"].value_counts()

self_employed    17476
salaried          6704
student           2058
retired           2024
company             40
Name: occupation, dtype: int64

In [13]:
customer_data["dependents"].fillna(0,inplace=True)
customer_data["occupation"].fillna('self_employed',inplace=True)
customer_data["city"].fillna(1020,inplace=True)

**Days Since Last Transaction**

In [14]:
customer_data["days_since_last_transaction"].describe()

count    25159.000000
mean        69.997814
std         86.341098
min          0.000000
25%         11.000000
50%         30.000000
75%         95.000000
max        365.000000
Name: days_since_last_transaction, dtype: float64

In [15]:
customer_data["days_since_last_transaction"].fillna(999,inplace=True)

**Encoding Categorical Variables**

In [16]:
customer_data = pd.get_dummies(customer_data)

In [17]:
customer_data["city"] = customer_data["city"].astype('object')
customer_data["branch_code"] = customer_data["branch_code"].astype('object')
customer_data["churn"] = customer_data["churn"].astype('category')

In [18]:
customer_data["dependents"].value_counts()

0.0     23898
2.0      2150
1.0      1395
3.0       701
4.0       179
5.0        41
6.0         8
7.0         3
36.0        1
52.0        1
25.0        1
9.0         1
50.0        1
32.0        1
8.0         1
Name: dependents, dtype: int64

In [19]:
customer_data["dependents"] = customer_data["dependents"].apply(lambda x: 8 if x > 8 else x)

In [20]:
num_cols = ['current_balance',
            'previous_month_end_balance', 'average_monthly_balance_prevQ2', 'average_monthly_balance_prevQ',
            'current_month_credit','previous_month_credit', 'current_month_debit', 
            'previous_month_debit','current_month_balance', 'previous_month_balance']
for i in num_cols:
    customer_data[i] = np.log1p(customer_data[i] + 17000)

std = StandardScaler()
scaled = std.fit_transform(customer_data[num_cols])
scaled = pd.DataFrame(scaled,columns=num_cols)

In [21]:
customer_data = customer_data.drop(columns = num_cols,axis = 1)
customer_data = customer_data.join(scaled) 

In [22]:
y = customer_data["churn"]
x = customer_data.drop(["customer_id","churn"],axis=1)

**Cross Validation**

In [23]:
# We can use class distribution in dataset to set threshold for predictions;

customer_data["churn"].value_counts() / customer_data.shape[0]

0    0.814671
1    0.185329
Name: churn, dtype: float64

In [24]:
from statistics import mean

def scorer(Model,threshold):
    kf = StratifiedKFold(n_splits=5,random_state=42,shuffle=True)
    roc_scores = [] 
    recall_scores = [] 
    precision_scores = []  
    
    for train_index,val_index in kf.split(x,y):
        xtr,xvl = x.loc[train_index],x.loc[val_index]
        ytr,yvl = y.loc[train_index],y.loc[val_index]
    
        model = Model
        model.fit(xtr,ytr)
        pred_prob = model.predict_proba(xvl)
        pred_class = []
    
        for j in pred_prob[:,1]:
            if j > threshold:
                pred_class.append(1)
            else:
                pred_class.append(0)
        
        pred_val = pred_class
        roc_score = metrics.roc_auc_score(yvl,pred_prob[:,1])
        recall = metrics.recall_score(yvl,pred_val)
        precision = metrics.precision_score(yvl,pred_val)
    
        roc_scores.append(roc_score)
        recall_scores.append(recall)
        precision_scores.append(precision)
    
    return mean(roc_scores), mean(recall_scores), mean(precision_scores)

In [25]:
scores_rf = scorer(RandomForestClassifier(),0.185)
scores_lr = scorer(LogisticRegression(),0.185)

In [26]:
print("Random Forest Model Cross Validataion Metrics :",)
print("Auc-Roc Score : " + str(scores_rf[0]), "Recall Score :" + str(scores_rf[1]),'\n')

print("Logistic Regression Model Cross Validation Metrics:")
print("Auc-Roc Score : " + str(scores_lr[0]), "Recall Score :" + str(scores_lr[1]),'\n')

Random Forest Model Cross Validataion Metrics :
Auc-Roc Score : 0.8346849794906871 Recall Score :0.7623574144486692 

Logistic Regression Model Cross Validation Metrics:
Auc-Roc Score : 0.7491168971456754 Recall Score :0.7129277566539924 



** Recall is a relevant metric to use in this case as the impact of false regatives is higher (if a customer is incorrectly identified to not churn, the bank might end up losing a customer even though it had predicted otherwise). So since we want to minimize number of false negatives, recall is a good metric to measure accuracy of our model. As per our best model, out of all customers who churn we would be able to predict about 75% of them accurately;**

**Model perfomance on unseen data**

In [27]:
xtrain, xtest, ytrain, ytest = train_test_split(x,y,test_size=0.30, random_state = 42)

model = RandomForestClassifier()
model.fit(xtrain,ytrain)
pp = model.predict_proba(xtest)[:,1]

# Using same threshold as validataion 
ypt = []

for p in pp:
    if p > 0.185:
        ypt.append(1)
    else:
        ypt.append(0)
    

roc_score = metrics.roc_auc_score(ytest,pp)
recall_score = metrics.recall_score(ytest,ypt)
precision_score = metrics.precision_score(ytest,ypt)

In [29]:
 # Performance on unseen data
    
print("Test Roc Score : " + str(roc_score), "Test Recall Score :" + str(recall_score))

Test Roc Score : 0.8353773919055094 Test Recall Score :0.7648578811369509


** The current model has good roc score with a decent recall score. This model can be used as a baseline and further optimized (hyperparameter tuning, top feature selction etc) to imporve overall metrics.**

*****************************